In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib

import sklearn.pipeline
import sklearn.compose
import sklearn.preprocessing
import sklearn.feature_extraction.text

In [ ]:
xgb.__version__

In [ ]:
! ls ../data

In [ ]:
! ls ../data/titanic

In [ ]:
df = pd.read_csv("../data/titanic/train.csv")
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

## Preprocess

In [ ]:
df.hist()

In [ ]:
df.loc[df['Cabin'].isnull(), 'Cabin'] = 'None'

In [ ]:
preprocessor = sklearn.compose.ColumnTransformer(
    transformers=[
        ('Pclass',   sklearn.preprocessing.FunctionTransformer(), ["Pclass"]), # identity
        ('Sex',      sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore'), ["Sex"]),
        ('Age',      sklearn.preprocessing.MinMaxScaler(), ["Age"]),
        ('SibSp',    sklearn.preprocessing.QuantileTransformer(), ["SibSp"]),
        ('Parch',    sklearn.preprocessing.FunctionTransformer(), ["Parch"]),
        ('Fare',     sklearn.preprocessing.Normalizer(), ["Fare"]),
        ('Cabin',    sklearn.preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=9999), ["Cabin"]),
        ('Embarked', sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore'), ['Embarked']),
    ],
)
tr = preprocessor.fit(df)

In [ ]:
preprocessed_df = tr.transform(df)
preprocessed_df[:10]

In [ ]:
preprocessed_df.shape

## Fit

In [ ]:
clf = xgb.XGBModel(**{'objective':'binary:logistic', 'n_estimators':10})

model = clf.fit(preprocessed_df, df['Survived'])

## Test

In [ ]:
y_pred = clf.predict(preprocessed_df)

In [ ]:
res = df['Survived'] - y_pred
res.abs().mean()

## Save Model

In [ ]:
joblib.dump(preprocessor, '../data/models/titanic_preprocessor.sklearn')
clf.save_model("../data/models/titanic.xgb")

In [ ]:
# test loaded model
preprocessor2 = joblib.load('../data/models/titanic_preprocessor.sklearn')
                            
clf2 = xgb.XGBModel(**{'objective':'binary:logistic', 'n_estimators':10})
clf2.load_model("../data/models/titanic.xgb")

In [ ]:
y_pred2 = clf2.predict(preprocessor2.transform(df))

res = df['Survived'] - y_pred
res.abs().mean()